In [21]:
import numpy as np
import pandas as pd
import re
import os

In [22]:
pd.set_option('display.max_columns', None)
data_raw = pd.read_csv("boat_data.csv")
data_raw

,Unnamed: 0,Hull Type,Rigging Type,LOA,LWL,Beam,S.A. (reported),Draft (max),Displacement,Ballast,S.A./Disp.,Bal./Disp.,Disp./Len.,Construction,Ballast Type,First Built,# Built,Designer,Comfort Ratio,Capsize Screening Formula,S#,I,J,P,E,SPL/TPS,ISP,S.A. Fore,S.A. Main,S.A./Disp. (calc.),Est. Forestay Len.,Download Boat Record,name,Draft (min),Builder,Model,Last Built,Make,Type,HP,Fuel,Headroom,Water,Website,KSP,PY,EY,Bridgedeck Clearance,Mast Height from DWL,Bruce Number
0,0,Fin w/bulb & spade rudder,Fractional Sloop,10.30 m,8.20 m,2.50 m,41.81 m<sup>2</sup>,1.80 m,"1,600 kg",725 kg,31.16,45.31,80.89,"FG w/core,Composite",Lead,1990.0,350.0,Ron Holland & Rolf Gyhlenius,11.41,2.16,6.48,9.81 m,3.10 m,11.00 m,4.15 m,4.00 m,12.50 m,15.22 m<sup>2</sup>,22.83 m<sup>2</sup>,28.36,10.29 m,"<a class="""" href=""https://sailboatdata.com/sai...",11 METER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Fin w/bulb & spade rudder,Fractional Sloop,7.32 m,5.49 m,1.27 m,14.59 m<sup>2</sup>,0.91 m,413 kg,136 kg,26.80,32.97,69.66,Plywood or GRP,Iron,1938.0,750.0,C. Raymond Hunt,10.58,1.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<a class="""" href=""https://sailboatdata.com/sai...",110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Centerboard Dinghy,Gunter,5.97 m,5.39 m,1.43 m,12.73 m<sup>2</sup>,0.29 m,231 kg,NaN,34.39,NaN,41.06,Wood,NaN,1931.0,NaN,Krogen Brothers,5.48,2.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<a class="""" href=""https://sailboatdata.com/sai...",12 SQUARE METER SHARPIE,0.97 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Centerboard Dinghy,Fractional Sloop,3.81 m,NaN,1.43 m,9.48 m<sup>2</sup>,0.58 m,50 kg,NaN,71.11,NaN,NaN,Plywood/FG,NaN,1971.0,3100.0,Simon Grieg,NaN,3.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<a class="""" href=""https://sailboatdata.com/sai...",125,0.08 m,See Class Association,Various,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Fin w/bulb & spade rudder,Fractional Sloop,10.67 m,9.60 m,3.28 m,74.51 m<sup>2</sup>,2.31 m,"2,971 kg","1,145 kg",36.77,38.55,93.55,FG,Lead,1998.0,48.0,Nelson/Marek,13.15,2.30,6.50,12.89 m,3.81 m,13.55 m,5.66 m,4.50 m,13.60 m,24.56 m<sup>2</sup>,38.34 m<sup>2</sup>,31.04,13.44 m,"<a class="""" href=""https://sailboatdata.com/sai...",1D 35,NaN,Carroll Marine Ltd.(USA),2GM20FC,2001.0,Yanmar,Diesel,18.5,45 L,1.62 m,159 L,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8770,8770,Fin w/spade rudder,Masthead Sloop,15.72 m,13.39 m,4.53 m,153.94 m<sup>2</sup>,2.35 m,"18,660 kg","7,036 kg",22.34,37.71,216.63,FG,Lead,1999.0,NaN,Studio Starkel,37.82,1.73,2.46,20.18 m,5.80 m,17.70 m,6.20 m,NaN,NaN,58.53 m<sup>2</sup>,54.87 m<sup>2</sup>,16.46,21.00 m,"<a class="""" href=""https://sailboatdata.com/sai...",ZUANELLI 49,NaN,Cantiere Zuanelli (ITA),NaN,NaN,Volvo Penta,Diesel,81,575 L,NaN,950 L,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8771,8771,Centerboard (Trunk),Fractional Sloop,5.79 m,5.31 m,1.88 m,14.96 m<sup>2</sup>,1.09 m,279 kg,NaN,35.64,NaN,52.02,Wood/FG,NaN,1960.0,NaN,Ernst Lehfeld,4.72,2.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<a class="""" href=""https://sailboatdata.com/sai...",ZUGVOGEL (CB),0.15 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8772,8772,Fin Keel w/bulb,Fractional Sloop,5.79 m,5.31 m,1.88 m,17.00 m<sup>2</sup>,0.86 m,379 kg,80 kg,33.05,21.05,70.60,Wood/FG,Lead,1960.0,NaN,Ernst Lehfeld,6.40,2.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<a class="""" href=""https://sailboatdata.com/sai...",ZUGVOGEL (KEEL),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8773,8773,Dbrd. Dinghy,Cat (unstayed),3.89 m,3.79 m,1.52 m,6.04 m<sup>2</sup>,0.66 m,59 kg,NaN,40.75,NaN,30.06,FG,NaN,1984.0,NaN,Gary Grossman/Steve Nichols,1.86,3.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [23]:
units_cleaner_tag = re.compile(r'([\d.]+)(\s*m|\s*kg)(<.*>)*')

In [24]:
def clean_row(row):
  cleaned_row = []
  for i in range(len(row)):
    if i != 32:
      try:
        cleaned_row.append(float(re.sub(units_cleaner_tag , r'\g<1>' , row[i].replace(',','') , )))
      except:
        cleaned_row.append(row[i])
    else:
      cleaned_row.append(row[i])
  return cleaned_row

In [25]:
cleaned_data = []
for i , row in data_raw.iterrows():
  cleaned_data.append(clean_row(row))

In [26]:
data_clean = pd.DataFrame(cleaned_data , columns = data_raw.columns)
try:
  data_clean.drop(columns = ['Unnamed: 0'] , inplace=True) # Remove an error I made when saving scraped data
except:
  pass

In [27]:
for i in range(0,len(data_clean)):
    path = "data/images/"+str(data_clean['name'][i]).replace(" ","_").replace("/","_")
    try:
        if len(os.listdir(path)) == 0:
            os.rmdir(path)
            data_clean.drop(axis = "index" , index=i , inplace=True)
        else:
            pass
    except:
        # print("No dir named" , path)
        data_clean.drop(axis = "index" , index=i , inplace=True)

data_clean.to_csv("boat_data_cleaned.csv" , index=False)


In [31]:
"data/images/"+str('COOPER 416 TM').replace(" ","_").replace("/","_")

'data/images/COOPER_416_TM'

In [28]:
data_clean.index[(data_clean['name'].eq('49ERFX'))].tolist()[0]

17

In [29]:
data_clean.__len__()

7088

In [30]:
pd.factorize(data_clean['Hull Type'])

(array([ 0,  0,  1, ..., 30,  2,  1], dtype=int64),
 Index(['Fin w/bulb & spade rudder', 'Centerboard Dinghy', 'Dbrd. Dinghy',
        'Fin w/spade rudder', 'Fin with rudder on skeg', 'Fin Keel',
        'Swing Keel', 'Centerboard (Trunk)', 'Keel/Cbrd.', 'Long Keel',
        'Full keel with attached rudder', 'Foiling Monohull',
        'Fin w/transom hung rudder', 'Lifting Keel', 'Catamaran Twin Dbrd.',
        'Scow Twin Cbrd.', 'Catamaran Twin Cbrd.', 'Catamaran Twin Keel',
        'Fin w/bulb & dual rudders', 'Fin w/bulb and transom hung rudder',
        'Twin Keel', 'Long keel w/trans. hung rudder', 'Modified Full Keel',
        'Keel/CB & spade rudder', 'Keel/CB w/dual rudders', 'Wing Keel',
        'Fin (shoal draft)', 'Full Keel', 'Fin w/rudder on partial skeg',
        'Catamaran Single Dbrd.', 'Fin Keel w/bulb', 'Cbrd w/outboard rudder',
        'Trimaran Dbrd.', 'Trimaran Cbrd.', 'Twin Centerboards', 'Triple Keel',
        'Cbrd (trunk) w/dual rudders', 'Sheel Keel',
        